Reference:  
 https://github.com/ethanjperez/pytorch-pretrained-BERT/tree/master/examples/lm_finetuning  
 https://github.com/ethanjperez/pytorch-pretrained-BERT/blob/master/examples/extract_features.py


In [7]:
!rm -rf NLP_Colab
!git clone https://github.com/sumkh/NLP_Colab.git

Cloning into 'NLP_Colab'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 36 (delta 15), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [0]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

In [9]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
!pip install pytorch-pretrained-bert pytorch-nlp -q

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel, BertAdam
from pytorch_pretrained_bert import modeling, tokenization
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

## Finetuning Bert Model

The LM finetuning code is an adaption to a script from the huggingface/pytorch-transformers repository:
* https://github.com/huggingface/pytorch-transformers/blob/v1.0.0/examples/lm_finetuning/finetune_on_pregenerated.py






Prepare the finetuning corpus, here shown for a test corpus "dev_corpus.txt":

    python pregenerate_training_data.py \
    --train_corpus **name_of_copus.txt** \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_prepared/ \
    --epochs_to_generate 2 --max_seq_len 256

Run actual finetuning with:

    python finetune_on_pregenerated.py \
    --pregenerated_data dev_corpus_prepared/ \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_finetuned/ \
    --epochs 2 --train_batch_size 16

In [13]:
!python NLP_Colab/pregenerate_training_data.py \
    --train_corpus NLP_Colab/transcript_corpus.txt \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_prepared/ \
    --epochs_to_generate 2 --max_seq_len 256

Loading Dataset: 115146 lines [00:16, 7085.65 lines/s]
Epoch:   0% 0/2 [00:00<?, ?it/s]
Document:   0% 0/56244 [00:00<?, ?it/s]
Document:   2% 997/56244 [00:00<00:05, 9968.61it/s]
Document:   4% 2036/56244 [00:00<00:05, 10090.69it/s]
Document:   6% 3167/56244 [00:00<00:05, 10427.14it/s]
Document:   8% 4230/56244 [00:00<00:04, 10486.85it/s]
Document:  10% 5347/56244 [00:00<00:04, 10681.51it/s]
Document:  11% 6461/56244 [00:00<00:04, 10813.66it/s]
Document:  13% 7578/56244 [00:00<00:04, 10915.90it/s]
Document:  15% 8684/56244 [00:00<00:04, 10957.87it/s]
Document:  17% 9814/56244 [00:00<00:04, 11057.48it/s]
Document:  19% 10881/56244 [00:01<00:04, 10884.66it/s]
Document:  21% 11944/56244 [00:01<00:04, 10805.54it/s]
Document:  23% 13028/56244 [00:01<00:03, 10815.50it/s]
Document:  25% 14097/56244 [00:01<00:03, 10622.89it/s]
Document:  27% 15210/56244 [00:01<00:03, 10768.84it/s]
Document:  29% 16328/56244 [00:01<00:03, 10887.16it/s]
Document:  31% 17438/56244 [00:01<00:03, 10947.83it/s]
Doc

In [14]:
# Copy the files to a directory in your Google Drive.
!cp -r ./dev_corpus_prepared/ "./drive/My Drive/EBAC_G/NLP_Project/Embeddings/"

# Zip up the folder and copy zip file to Google Drive
!zip -r /content/dev_corpus_prepared.zip /content/dev_corpus_prepared
!cp -r /content/dev_corpus_prepared.zip "./drive/My Drive/EBAC_G/NLP_Project/Embeddings/"


  adding: content/dev_corpus_prepared/ (stored 0%)
  adding: content/dev_corpus_prepared/epoch_1.json (deflated 81%)
  adding: content/dev_corpus_prepared/epoch_0.json (deflated 81%)
  adding: content/dev_corpus_prepared/epoch_0_metrics.json (stored 0%)
  adding: content/dev_corpus_prepared/epoch_1_metrics.json (stored 0%)


In [15]:
!python NLP_Colab/finetune_on_pregenerated.py \
    --pregenerated_data dev_corpus_prepared/ \
    --bert_model bert-base-uncased --do_lower_case \
    --output_dir dev_corpus_finetuned/ \
    --epochs 2 --train_batch_size 16

2020-03-08 12:28:44,030: device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
2020-03-08 12:28:44,309: loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2020-03-08 12:28:44,749: loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
2020-03-08 12:28:44,750: extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp9duwzvvv
2020-03-08 12:28:48,406: Model config {
  "attention_probs

#### Download Fine-Tuned BERT Model

In [16]:
!ls -l --block-size=K ./dev_corpus_finetuned/

total 430388K
-rw-r--r-- 1 root root      1K Mar  8 13:39 config.json
-rw-r--r-- 1 root root 430152K Mar  8 13:39 pytorch_model.bin
-rw-r--r-- 1 root root    227K Mar  8 13:39 vocab.txt


In [17]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./dev_corpus_finetuned/ "./drive/My Drive/EBAC_G/NLP_Project/Embeddings/"

# Zip up the folder and copy zip file to Google Drive
!zip -r /content/dev_corpus_finetuned.zip /content/dev_corpus_finetuned
!cp -r /content/dev_corpus_finetuned.zip "./drive/My Drive/EBAC_G/NLP_Project/Embeddings/"

#If want to download from google.colab import files (slower) 
#files.download("/content/dev_corpus_finetuned.zip")

  adding: content/dev_corpus_finetuned/ (stored 0%)
  adding: content/dev_corpus_finetuned/pytorch_model.bin (deflated 7%)
  adding: content/dev_corpus_finetuned/vocab.txt (deflated 53%)
  adding: content/dev_corpus_finetuned/config.json (deflated 47%)


## Extract Words Embeddings with Bert-Base-Uncased (Examples)

As an example, we include the script extract_features.py which can be used like this:  
https://github.com/ethanjperez/pytorch-pretrained-BERT/blob/master/examples/extract_features.py  

**Please use the one from my NLP_Colab Github respository which is an adapted version.**

In [0]:
# Sentence A and Sentence B are separated by the ||| delimiter for sentence
# pair tasks like question answering and entailment.
# For single sentence inputs, put one sentence per line and DON'T use the
# delimiter.

!echo 'Who was Jim Henson ? ||| Jim Henson was a puppeteer' > input.txt

# https://github.com/google-research/bert/blob/master/README.md
# Download BERT Model to your drive:
# https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

input_file="./input.txt"
output_file="./bert_base_WordVectors.json"

bert_master="./NLP_Colab"

!python $bert_master/extract_features.py \
  --input_file=$input_file \
  --output_file=$output_file \
  --bert_model bert-base-uncased --do_lower_case \
  --layers=-1,-2,-3,-4 \
  --max_seq_length=128 \
  --batch_size=8

In [0]:
# Zip up the folder and copy zip file to Google Drive
!zip -r /content/bertwordvector.zip /content/bertWordVectors.json
!cp -r /content/bertwordvector.zip "./drive/My Drive/Downloads"

  adding: content/bertWordVectors.jsonl (deflated 63%)


# Using Fine-Tune Model

In [11]:
# Load a trained model and vocabulary that you have fine-tuned
#output_dir = './model_save/'
output_dir = './drive/My Drive/EBAC_G/NLP_Project/Embeddings/dev_corpus_finetuned/'
#model = model_class.from_pretrained(output_dir)
model = BertModel.from_pretrained(output_dir)
#tokenizer = tokenizer_class.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [0]:
#!echo 'Who was Jim Henson ? ||| Jim Henson was a puppeteer' > input.txt
#input_file="./input.txt"
input_file="NLP_Colab/transcript_corpus_healthcare.txt"
layer_indexes = [-1,-2,-3,-4]
batch_size = 32 
local_rank = -1 #local_rank for distributed training on gpus

#### Import Packages

In [0]:
import argparse
import collections
import logging
import json
import re

import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


#### read_examples(input_file)
**Return examples**  
example.InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))

In [0]:
class InputExample(object):

    def __init__(self, unique_id, text_a, text_b):
        self.unique_id = unique_id
        self.text_a = text_a
        self.text_b = text_b

examples = []
unique_id = 0
with open(input_file, "r", encoding='utf-8') as reader:
    while True:
        line = reader.readline()
        if not line:
            break
        line = line.strip()
        text_a = None
        text_b = None
        m = re.match(r"^(.*) \|\|\| (.*)$", line)
        if m is None:
            text_a = line
        else:
            text_a = m.group(1)
            text_b = m.group(2)
            
        examples.append(InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
        unique_id += 1


In [16]:
print(examples[1].unique_id)
print(examples[1].text_a)
print(examples[1].text_b)

1
Today I'll highlight recent pipeline updates and discuss some of the key milestones we anticipate for the remainder of 2017.
None


#### convert_examples_to_features(examples, seq_length, tokenizer)  
**Return features**   
features.InputFeatures(unique_id=example.unique_id, 
tokens=tokens,
input_ids=input_ids,
input_mask=input_mask,
input_type_ids=input_type_ids))

In [17]:
seq_length = 128 
# type=int
# The maximum total input sequence length after WordPiece tokenization. 
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
        self.unique_id = unique_id
        self.tokens = tokens
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.input_type_ids = input_type_ids

features = []
for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > seq_length - 2:
            tokens_a = tokens_a[0:(seq_length - 2)]

    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            input_type_ids.append(1)
        tokens.append("[SEP]")
        input_type_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < seq_length:
        input_ids.append(0)
        input_mask.append(0)
        input_type_ids.append(0)

    assert len(input_ids) == seq_length
    assert len(input_mask) == seq_length
    assert len(input_type_ids) == seq_length

    if ex_index < 5:
        logger.info("*** Example ***")
        logger.info("unique_id: %s" % (example.unique_id))
        logger.info("tokens: %s" % " ".join([str(x) for x in tokens]))
        logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        logger.info(
            "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))
        
    features.append(InputFeatures(
                unique_id=example.unique_id,
                tokens=tokens,
                input_ids=input_ids,
                input_mask=input_mask,
                input_type_ids=input_type_ids))

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


03/08/2020 10:11:12 - INFO - __main__ -   *** Example ***
03/08/2020 10:11:12 - INFO - __main__ -   unique_id: 0
03/08/2020 10:11:12 - INFO - __main__ -   tokens: [CLS] [SEP]
03/08/2020 10:11:12 - INFO - __main__ -   input_ids: 101 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/08/2020 10:11:12 - INFO - __main__ -   input_mask: 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/08/2020 10:11:12 - INFO - __main__ -   input_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [19]:
print(features[1].unique_id)
print(features[1].tokens)
print(features[1].input_ids)
print(features[1].input_mask)
print(features[1].input_type_ids)

1
['[CLS]', 'today', 'i', "'", 'll', 'highlight', 'recent', 'pipeline', 'updates', 'and', 'discuss', 'some', 'of', 'the', 'key', 'milestone', '##s', 'we', 'anti', '##ci', '##pate', 'for', 'the', 'remainder', 'of', '2017', '.', '[SEP]']
[101, 2651, 1045, 1005, 2222, 12944, 3522, 13117, 14409, 1998, 6848, 2070, 1997, 1996, 3145, 19199, 2015, 2057, 3424, 6895, 17585, 2005, 1996, 6893, 1997, 2418, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

#### Function to Extract Words Embeddings

In [0]:
unique_id_to_feature = {}
for feature in features:
    unique_id_to_feature[feature.unique_id] = feature

if local_rank != -1:
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[local_rank], output_device=local_rank)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)

eval_data = TensorDataset(all_input_ids, all_input_mask, all_example_index)
if local_rank == -1:
    eval_sampler = SequentialSampler(eval_data)
else:
    eval_sampler = DistributedSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=batch_size)

model.eval()

for input_ids, input_mask, example_indices in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)

    all_encoder_layers, _ = model(input_ids, token_type_ids=None, attention_mask=input_mask)
    all_encoder_layers = all_encoder_layers

    for b, example_index in enumerate(example_indices):
        feature = features[example_index.item()]
        unique_id = int(feature.unique_id)
        # feature = unique_id_to_feature[unique_id]
        output_json = collections.OrderedDict()
        output_json["linex_index"] = unique_id
        all_out_features = []
        for (i, token) in enumerate(feature.tokens):
            all_layers = []
            for (j, layer_index) in enumerate(layer_indexes):
                layer_output = all_encoder_layers[int(layer_index)].detach().cpu().numpy()
                layer_output = layer_output[b]
                layers = collections.OrderedDict()
                layers["index"] = layer_index
                layers["values"] = [
                    round(x.item(), 6) for x in layer_output[i]
                ]
                all_layers.append(layers)
            out_features = collections.OrderedDict()
            out_features["token"] = token
            out_features["layers"] = all_layers
            all_out_features.append(out_features)
        output_json["features"] = all_out_features
            


In [21]:
print(json.dumps(output_json))

{"linex_index": 1079, "features": [{"token": "[CLS]", "layers": [{"index": -1, "values": [0.411986, -0.522559, 0.285365, -0.007502, 0.851679, 0.193578, -0.650298, -0.61432, 0.201131, -0.508109, -0.087956, 0.369519, -0.10683, -0.544949, -0.419571, 0.115838, 0.276748, -0.402175, -0.414788, -0.097323, -0.704712, -0.091555, 0.130447, 0.200553, 0.140762, 0.118146, 0.266468, 0.070276, -0.333373, -0.266885, 0.13556, -0.665268, 0.140506, -0.172081, -0.539581, 0.162969, 0.206219, 0.432521, -0.29902, -0.177419, -0.12403, -0.163031, -0.448793, 0.040709, -0.217324, 0.658567, -2.565645, -0.20935, 0.525307, -0.09042, 0.861792, -0.071082, -0.373387, 0.502678, 0.335402, -0.505556, 0.093757, 0.327967, -0.620222, -0.630457, 0.289342, -0.119191, 0.645752, 0.378842, 0.581308, -0.685508, -0.000213, -0.412824, 0.012869, 0.249945, 0.370998, 0.137252, -0.610399, -0.060675, 0.106094, -0.052704, 0.106082, -0.123949, 0.156393, 0.660304, -0.238605, -0.402977, -0.56757, -0.482814, -0.789666, -0.676484, 0.155847, 0

In [0]:
with open("bertWordVectors.json", "w", encoding='utf-8') as writer:
  writer.write(json.dumps(output_json) + "\n")

In [100]:
# Zip up the folder and copy zip file to Google Drive
!zip -r /content/bertwordvector.zip /content/bertWordVectors.json
!cp -r /content/bertwordvector.zip "./drive/My Drive/Downloads"

  adding: content/bertWordVectors.json (deflated 62%)
